In [6]:
%pip install -e ..

Obtaining file:///C:/Users/frolo/Documentos/PythonProjects/forecast-models
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/111.9 kB ? eta -:--:--
     --- ------------------------------------ 10.2/111.9 kB ? eta -:--:--
     ----------------------- ------------- 71.7/111.9 kB 787.7 kB/s eta 0:00:01
     -------------------------------------- 111.9/111.9 kB 1.1 


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\frolo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
# add repo root and /src to the path
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../src"))

# smoke test
from forecast_models.data.loaders import load_timeseries
from forecast_models.data.transforms import adf_unit_root_test, choose_transform_auto, StationarityTransform
from forecast_models.models.arima import ARIMA, ARIMAConfig, AutoARIMA, AutoARIMAConfig
print("import ok")

import ok


In [ ]:
# 1) Load (no manual differencing/log here; let AutoARIMA handle stationarity)
df = load_timeseries(
    "entsoe_day_ahead_price_10YES-REE------0.parquet",
    time_col="timestamp_utc",
    value_col="value",
)

N_DAYS = 120  # try 60–180
df = df.tail(N_DAYS * 24).reset_index(drop=True)

# 2) Pick sensible seasonality for your data
# For hourly prices with daily pattern:
m = 24

cfg = AutoARIMAConfig(
    seasonal=True, m=m,
    max_p=3, max_q=3, max_P=1, max_Q=1, max_order=6,
    test="adf", seasonal_test="ocsb",
    information_criterion="aic",
    use_boxcox="auto",
    stepwise=True, trace=False, suppress_warnings=True, error_action="ignore",
    trend=None, with_intercept=False, simple_differencing=True
)

model = AutoARIMA(cfg)
model.fit(df["y"].values)

H = 24
yhat = np.asarray(model.predict(H), dtype=float)

# Build future index
freq = pd.infer_freq(df["ds"])
if freq is None:
    # Fallback: assume hourly
    freq = "H"
last_ts = df["ds"].iloc[-1]
future_index = pd.date_range(
    start=last_ts + pd.tseries.frequencies.to_offset(freq),
    periods=H, freq=freq, tz="UTC"
)
pred_df = pd.DataFrame({"ds": future_index, "yhat": yhat})

# Plot last 96h + forecast 24h
hist_tail = df.tail(96)

plt.figure(figsize=(10, 4))
plt.plot(hist_tail["ds"], hist_tail["y"], label="History (last 96h)")
plt.plot(pred_df["ds"], pred_df["yhat"], label="AutoARIMA forecast (24h)")
plt.title("Day-ahead price: last 96h + 24h AutoARIMA forecast")
plt.xlabel("Time (UTC)")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()

# Quick sanity
print(f"History tail: {hist_tail['ds'].iloc[0]} → {hist_tail['ds'].iloc[-1]}")
print(f"Forecast:     {pred_df['ds'].iloc[0]} → {pred_df['ds'].iloc[-1]}")
print("First 5 preds:", pred_df.head().to_dict(orient="records"))


C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Pyt

MemoryError: Unable to allocate 857. MiB for an array with shape (49, 49, 46775) and data type float64